## Main Notebook for hyperparameter tunning

In [1]:
%load_ext autoreload
%autoreload 1

%aimport dataset_processing
import pandas as pd
import numpy as np
import datetime as dt

training_path = "data/training_dataset.csv"


In [4]:
%aimport models

In [2]:
train_df = dataset_processing.generate_dataset(training_path,interval_length=3,
                                               num_past_intervals=15,num_timesteps_whole_network = 0,
                                                 exclude_zero_flows=True, start_time = dt.time(7,30),
                                                 end_time = dt.time(8,30))

In [7]:
red_train_df = train_df[train_df["datetime"].dt.day >= 10]
val_df =train_df[train_df["datetime"].dt.day < 10]

In [13]:
## Train Linear Regression on train set and test on validation set
model_lr, rmse_train_lr, rmse_test_lr, mape_train_lr, mape_test_lr = models.train_linear_regression(red_train_df, val_df)
print("RMSE Train: ", rmse_train_lr)
print("RMSE Test: ", rmse_test_lr)
print("MAPE Train: ", mape_train_lr)
print("MAPE Test: ", mape_test_lr)

RMSE Train:  23.159396080334027
RMSE Test:  23.873260644895133
MAPE Train:  0.09887361519457404
MAPE Test:  0.0946018945082217


### Select best hyperparameters for LASSO Regression

In [12]:
alpha_range = [0.001,0.01,0.05,0.1,0.2,0.3,0.4]

results_lasso_all_sensors_df = pd.DataFrame(columns=[
    "alpha",
    "rmse_train_lasso", "rmse_test_lasso", "mape_train_lasso", "mape_test_lasso",
])

for alpha in alpha_range:
    model_dict_lasso, rmse_train_lasso, rmse_test_lasso, mape_train_lasso, mape_test_lasso = models.train_lasso_regression(red_train_df, val_df,alpha)
    results_lasso_all_sensors_df = pd.concat([
    results_lasso_all_sensors_df,
    pd.DataFrame([{
        "alpha": alpha,
        "rmse_train_lasso": rmse_train_lasso, "rmse_test_lasso": rmse_test_lasso,
        "mape_train_lasso": mape_train_lasso, "mape_test_lasso": mape_test_lasso,
    }])
], ignore_index=True)
    
results_lasso_all_sensors_df        


d:\KTH\Applied Artificial Intelligence in Transportation\AH2179-Project\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.791e+06, tolerance: 2.113e+05
  model = cd_fast.enet_coordinate_descent(
C:\Users\jemaa\AppData\Local\Temp\ipykernel_31056\740915291.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_lasso_all_sensors_df = pd.concat([


,alpha,rmse_train_lasso,rmse_test_lasso,mape_train_lasso,mape_test_lasso
0,0.001,23.159431,23.876099,0.098854,0.094610
1,0.010,23.162806,23.903968,0.098706,0.094705
2,0.050,23.222214,24.034239,0.098585,0.095344
3,0.100,23.318855,24.145766,0.098827,0.095753
4,0.200,23.518044,24.338993,0.099632,0.096496
5,0.300,23.613888,24.428799,0.100058,0.096919
6,0.400,23.672308,24.473535,0.100330,0.097193


In [ ]:

alpha_range = [0.001,0.01,0.05,0.1,0.2,0.3,0.4]

results_lasso_df = pd.DataFrame(columns=[
    "alpha",
    "rmse_train_lasso", "rmse_test_lasso", "mape_train_lasso", "mape_test_lasso",
])

for alpha in alpha_range:
    model_dict_lasso, rmse_train_lasso, rmse_test_lasso, mape_train_lasso, mape_test_lasso = models.train_sensor_models_with_lasso(red_train_df, val_df,alpha)
    results_lasso_df = pd.concat([
    results_lasso_df,
    pd.DataFrame([{
        "alpha": alpha,
        "rmse_train_lasso": rmse_train_lasso, "rmse_test_lasso": rmse_test_lasso,
        "mape_train_lasso": mape_train_lasso, "mape_test_lasso": mape_test_lasso,
    }])
], ignore_index=True)
    
results_lasso_df        


d:\KTH\Applied Artificial Intelligence in Transportation\AH2179-Project\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.520e+05, tolerance: 4.162e+03
  model = cd_fast.enet_coordinate_descent(
d:\KTH\Applied Artificial Intelligence in Transportation\AH2179-Project\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.168e+05, tolerance: 3.505e+03
  model = cd_fast.enet_coordinate_descent(
d:\KTH\Applied Artificial Intelligence in Transportation\AH2179-Project\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converg

,alpha,rmse_train_lasso,rmse_test_lasso,mape_train_lasso,mape_test_lasso
0,0.001,22.040449,23.247203,0.091326,0.088635
1,0.010,22.040722,23.240747,0.091326,0.088618
2,0.050,22.046280,23.218464,0.091358,0.088588
3,0.100,22.060211,23.199486,0.091452,0.088587
4,0.200,22.095526,23.179555,0.091714,0.088686
5,0.300,22.140182,23.178126,0.092023,0.088849
6,0.400,22.190258,23.184882,0.092345,0.089041


### Perform hyperparamter tunning for neural network

In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

interval_features = train_df.filter(like="bin_interval_").columns.to_list()
flow_features = train_df.filter(regex=r"^flow_prev").columns.to_list()
dummy_vars_dow = train_df.filter(like="dow_").columns.to_list()
dummy_vars_dp_id = train_df.filter(like="sensor_id").columns.to_list()
dummy_vars_portal = train_df.filter(like="portal__").columns.to_list()


train_features = flow_features + dummy_vars_dow + ["is_weekend"] + interval_features + \
                    dummy_vars_dp_id + dummy_vars_portal

target = "flow_next_15min"

X_train = red_train_df[train_features].to_numpy()
y_train = red_train_df[target].to_numpy()

X_val = val_df[train_features].to_numpy()
y_val = val_df[target].to_numpy()

number_neurons = [32,64,128]
number_layers = [1,2,3]
grid_search_df = pd.DataFrame(columns=["number_neurons","number_layers","train_rmse", "val_rmse", "train_mape","val_mape"])

# Test all neurons and layers combinations
for neurons in number_neurons: 
    for num_layer in number_layers: 
            model = Sequential()
            model.add(Dense(neurons, activation='relu',input_shape=(X_train.shape[1],)))

            for _ in range(num_layer):
                model.add(Dense(neurons, activation='relu'))

            model.add(Dense(1))
            opt = keras.optimizers.Adam(learning_rate=0.01)
            model.compile(optimizer=opt, loss='mse', 
                        metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
            model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)

            y_train_pred = model.predict(X_train, verbose=0)
            rmse_train = root_mean_squared_error(y_train, y_train_pred)
            mape_train = mean_absolute_percentage_error(y_train, y_train_pred)

            y_test_pred = model.predict(X_val,verbose=0)
            rmse_test = root_mean_squared_error(y_val, y_test_pred)
            mape_test = mean_absolute_percentage_error(y_val, y_test_pred)
            it_df = pd.DataFrame(data = {"number_neurons": neurons, "number_layers": num_layer,
                                            "train_rmse": rmse_train, "val_rmse": rmse_test,
                                            "train_mape": mape_train, "val_mape": mape_test}, index=[0])

            grid_search_df = pd.concat([grid_search_df, it_df], axis=0)


grid_search_df


d:\KTH\Applied Artificial Intelligence in Transportation\AH2179-Project\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\jemaa\AppData\Local\Temp\ipykernel_31056\1290624760.py:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  grid_search_df = pd.concat([grid_search_df, it_df], axis=0)
d:\KTH\Applied Artificial Intelligence in Transportation\AH2179-Project\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argu

,number_neurons,number_layers,train_rmse,val_rmse,train_mape,val_mape
0,32,1,22.384907,24.119580,0.089402,0.089247
0,32,2,22.073943,22.978031,0.088912,0.087487
0,32,3,22.586662,23.650941,0.093940,0.091361
0,64,1,22.105816,23.060241,0.090054,0.089075
0,64,2,22.742076,23.802472,0.090491,0.089332
0,64,3,23.179332,24.731155,0.091377,0.091928
0,128,1,22.152505,23.276241,0.088239,0.088314
0,128,2,22.891268,23.330788,0.093501,0.090866
0,128,3,23.317882,24.737050,0.091357,0.091119


In [ ]:
## Continue hyperparamter tunning but for dropout and L1 and L2 reglularization with a fixed number of layers and neurons

from keras.layers import Dropout
from keras.regularizers import L1L2
grid_search_dropout_df = pd.DataFrame(columns=["dropout","l1_value","l2_value" ,"train_rmse", "val_rmse", "train_mape","val_mape"])

dropout_values = [0.01,0.05,0.1,0.2]
l1_regularization_list = [0,0.01,0.05] 
l2_regularization_list = [0,0.01,0.05] 

for dropout in dropout_values: 
    for l1_value in l1_regularization_list:
        for l2_value in l2_regularization_list:
            model = Sequential()
            model.add(Dense(64, activation='relu',input_shape=(X_train.shape[1],), kernel_regularizer=L1L2(l1=l1_value,l2=l2_value)))
            model.add(Dropout(dropout))
            model.add(Dense(64, activation='relu', kernel_regularizer=L1L2(l1=l1_value,l2=l2_value)))
            model.add(Dropout(dropout))
            model.add(Dense(1))
            opt = keras.optimizers.Adam(learning_rate=0.01)
            model.compile(optimizer=opt, loss='mse', 
                        metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
            model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)

            y_train_pred = model.predict(X_train, verbose=0)
            rmse_train = root_mean_squared_error(y_train, y_train_pred)
            mape_train = mean_absolute_percentage_error(y_train, y_train_pred)

            y_test_pred = model.predict(X_val,verbose=0)
            rmse_test = root_mean_squared_error(y_val, y_test_pred)
            mape_test = mean_absolute_percentage_error(y_val, y_test_pred)
            it_df = pd.DataFrame(data = {"dropout": dropout, "l1_value": l1_value, "l2_value": l2_value,
                                        "train_rmse": rmse_train, "val_rmse": rmse_test,
                                        "train_mape": mape_train, "val_mape": mape_test}, index=[0])

            grid_search_dropout_df = pd.concat([grid_search_dropout_df, it_df], axis=0)


grid_search_dropout_df


d:\KTH\Applied Artificial Intelligence in Transportation\AH2179-Project\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\jemaa\AppData\Local\Temp\ipykernel_31056\1213986345.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  grid_search_dropout_df = pd.concat([grid_search_dropout_df, it_df], axis=0)
d:\KTH\Applied Artificial Intelligence in Transportation\AH2179-Project\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/

,dropout,l1_value,l2_value,train_rmse,val_rmse,train_mape,val_mape,droput
0,NaN,0,0,22.932096,23.355045,0.095618,0.092136,0.01
0,NaN,0,0.01,24.055433,24.353203,0.116182,0.110411,0.01
0,NaN,0,0.05,30.340425,30.325630,0.131806,0.125846,0.01
0,NaN,0.01,0,22.528833,23.131444,0.092338,0.090041,0.01
0,NaN,0.01,0.01,22.654436,24.050036,0.090440,0.089366,0.01
0,NaN,0.01,0.05,31.198999,32.940413,0.119014,0.119346,0.01
0,NaN,0.05,0,24.604793,25.897297,0.100276,0.098326,0.01
0,NaN,0.05,0.01,24.656870,25.838120,0.100749,0.098697,0.01
0,NaN,0.05,0.05,23.517972,24.187142,0.100345,0.096916,0.01
0,NaN,0,0,27.949948,29.529985,0.105113,0.106497,0.05
